# マーケットプレイスのデータを使った分析体験
- ハンズオン1: マーケットプレイスからデータを取得（[Finance & Economics](https://app.snowflake.com/marketplace/listing/GZTSZAS2KF7/snowflake-public-data-products-finance-economics?search=snowflake+finance)と[SEC Filings](https://app.snowflake.com/marketplace/listing/GZTSZAS2KH9/snowflake-public-data-products-sec-filings?search=snowflake+finance)）
- ハンズオン2: マーケットプレイスのデータを活用し Snowflake に関する情報を抽出
- ハンズオン3: Streamlit を使って加工したデータを可視化し示唆を得る
- ハンズオン4: LLM を使った分析

![img](https://lh3.googleusercontent.com/pw/AP1GczOMsqMbvn9RTHy22dHn2Akt-Kl-B3MCxyWbPynuYkv0ST0sPokttpzmC9mzxZV334QnSVr6C6tefW4EBMpGT8z8LrRcZHTSZsbEJ2HDfJrn4eMUD1dPh1j2-sI2o2qn-GlnKbmHPqsE32DObqxcFO1z=w2168-h1218-s-no-gm?authuser=0)

## 事前準備
-  パッケージより`modin=0.30.1`, `matplotlib`, `ploty`をインストール

## ハンズオン2. マーケットプレイスのデータを活用し Snowflake に関する情報を抽出
![img](https://lh3.googleusercontent.com/pw/AP1GczP1fJfALSTHJcSgBQlPU1B9SJ_UwZdIIDqzA1FjkBKcO7ZcRjmlAtYHiTUa3cUlw2cr9ySloEeyorcWaEat-GmHpT7d4AeUTyEzGhhRFD-KW5de6Lo12IMPh8H_LFesI6CQhlL_lviDI4PEPq7Mv0B5=w2264-h1274-s-no-gm?authuser=0)

データのチェック

In [ ]:
SELECT *
FROM FINANCE__ECONOMICS.CYBERSYN.STOCK_PRICE_TIMESERIES 
WHERE ticker IN ('SNOW')
ORDER BY ticker, date desc
LIMIT 10;

### 1. 様々な分析を試してみる
![img](https://lh3.googleusercontent.com/pw/AP1GczN-I3Qt6yn9KDaT-S5QwmsONdq7NxU14TlSUfcqECxprSmkzDYSIarA8Xv8gOd2rlvFrCjH3wZ79PyyS99oXjK_C6kScnqrvQL5Q3P4QkgIZfPIVQxq7Yf8M52EGMgp7NWx8rae5oi0bh_jzq_eiKi0=w1337-h750-s-no-gm?authuser=0)

#### TIME_SLICE を使った 個別銘柄の のパフォーマンス調査
![img](https://lh3.googleusercontent.com/pw/AP1GczNZfBn8uy_ynf_oLFdat4KGwl1sextzB-QUzH6vBCL-JnMkWRpWOj03AUeCAvYFX_eDemnJ_QtrIq0YHUcqRKGbPJZpn7UBC7WYCDqnhMEc0Cb6ib-Ia7TVPshmfIzrX3xbHS9d_mLPYBZJi232ivba=w1149-h646-s-no-gm?authuser=0)
- TIME_SLICE 関数を使用して、1週間の平均価格、取引日数、平均最安値、平均最高値を抽出するSQLを簡単に作成
- TIME_SLICE 関数は指定した時間単位に基づいて、タイムスタンプを時間、日、週などの一貫した間隔にデータを分割することが可能
- ユースケース: 特定の銘柄の平均価格や取引量の増減などの傾向を特定など

In [ ]:
-- 1週間ごとのSnowflakeの平均取引価格のデータを取得
SELECT
    ticker,
    TIME_SLICE(date, 1, 'WEEK', 'START') AS week_starting,
    AVG(value) AS average_price,
    COUNT(*) AS trading_days_count,
    MIN(value) AS week_min_price,
    MAX(value) AS week_max_price
FROM FINANCE__ECONOMICS.CYBERSYN.STOCK_PRICE_TIMESERIES 
WHERE variable_name = 'Post-Market Close' 
    AND ticker = 'SNOW'
    AND date IS NOT NULL
    AND value IS NOT NULL
GROUP BY TIME_SLICE(date, 1, 'WEEK', 'START'), ticker
ORDER BY week_starting DESC;

#### [ASOF JOIN](https://docs.snowflake.com/en/sql-reference/constructs/asof-join) を使用してASOF JOINを使った始値と終値の比較分析
![img](https://lh3.googleusercontent.com/pw/AP1GczMa3ZfE5UMdXkJc3_-WBLYella4DFrxRJxKg8X6tv-47geHSfT2gFupb9p_kDrFmgr53LL7QCXu3Pf-ONCpV4MRf0ZPdl2m1fdeODo6GWwAxMYNb19AlaO64dUEacDBrtskFYMjCFv7h4-w4wyEyGjq=w1149-h646-s-no-gm?authuser=0)

ASOF JOINの利点:
1. 完全一致しない時系列データでも効率的に結合
2. 「最も近い過去の値」を自動で取得
3. 複雑なサブクエリやウィンドウ関数が不要

例）
- 始値データ: 2025-06-27 09:30:15  $180.50
- 終値データ: 2025-06-27 16:00:23  $185.20

→ これらを通常のJOINでは結合できない （`本ハンズオンのデータは日付単位なので通常のJOINが可能`）

In [ ]:
-- 始値データを取得
WITH opening_prices AS (
    SELECT 
        date,
        ticker,
        value AS opening_price
    FROM FINANCE__ECONOMICS.CYBERSYN.STOCK_PRICE_TIMESERIES
    WHERE variable_name = 'Pre-Market Open'  -- 始値データ
        AND ticker = 'SNOW'                  -- Snowflake株のみ
        AND value IS NOT NULL                -- NULL値を除外
),

-- 終値データを取得
closing_prices AS (
    SELECT 
        date,
        ticker,
        value AS closing_price
    FROM FINANCE__ECONOMICS.CYBERSYN.STOCK_PRICE_TIMESERIES
    WHERE variable_name = 'Post-Market Close'  -- 終値データ
        AND ticker = 'SNOW'                    -- Snowflake株のみ
        AND value IS NOT NULL                  -- NULL値を除外
)

-- ASOF JOINで始値と終値を結合
-- 通常のJOINと違い、完全一致しなくても「最も近い過去の値」で結合
SELECT 
    c.date AS trade_date,
    c.ticker,
    o.opening_price,
    c.closing_price,
    -- 日次変動額を計算
    ROUND(c.closing_price - o.opening_price, 2) AS daily_change,
    -- 日次変動率（%）を計算
    ROUND((c.closing_price - o.opening_price) / o.opening_price * 100, 2) AS daily_change_percent
FROM closing_prices c
-- ASOF JOIN: 終値の日付以前で最も近い始値を取得
ASOF JOIN opening_prices o 
    MATCH_CONDITION (c.date >= o.date)  -- 終値日以前の始値を検索
    ON c.ticker = o.ticker              -- 同じ銘柄で結合
ORDER BY c.date DESC
LIMIT 30;  -- 最新30日分のデータを表示

#### SQLコマンドのチェーン実行で表現
![img](https://lh3.googleusercontent.com/pw/AP1GczM4MG6knr5nphq1llGUHB9nMYxPl72P2TxO90W_OCuTNKz2cqdwCmAFzQt2CBN_j7xuA8VdODz1tW6XU-o5zH_xPHp9v9AxtvkqhZ4KQIMlDkPF0vZfJTRAd6DqSvpC3nPX5DoHyiVY3PNrxg77xStO=w1149-h646-s-no-gm?authuser=0)
- 可読性と柔軟性を維持しながら、SQLクエリを直感的かつ明確に連結可能な新しい記述方式
- 依存関係のあるSQL操作やパイプラインを簡潔に表現可能な新機能
- 直前のSQLステートメントの結果には $n 構文でアクセス

In [ ]:
-- 第1ステップ：基本データの抽出
SELECT 
    ticker,
    asset_class,
    primary_exchange_code,
    primary_exchange_name,
    date,
    variable_name,
    value
FROM FINANCE__ECONOMICS.CYBERSYN.STOCK_PRICE_TIMESERIES 
WHERE ticker = 'SNOW'
AND variable_name IN ('Nasdaq Volume', 'Post-Market Close')

-- 第2ステップ：PIVOT操作の適用
->> SELECT *
    FROM $1
    PIVOT(
        MAX(value) 
        FOR variable_name IN (
            'Nasdaq Volume',
            'Post-Market Close'
        )
    )

-- 第3ステップ：結果の並び替え
->> SELECT * FROM $1
    ORDER BY date DESC;

補足）

![img](https://lh3.googleusercontent.com/pw/AP1GczPgbBPNUmDtaRekOnyQnyQU-xGTFKPkE0NnvbSGsLBEyFbGYAUXqa9jw7BBQblRUjSQ2py0qPajNIrpFKDCaHQHEfXhD1pAlXkZCsd_er4QRGf10dnEd-G23EX7jGS5gEzaulDmwMCWQp5If9AhEzPm=w1569-h880-s-no-gm?authuser=0)

#### 参考）Snowpark Pythonを使った表現
- Snowpark Python は、Snowflake 上で「データを外に持ち出さずに」Python コードを実行できる開発フレームワーク
- つまりSnowflake 内で完結する大規模データ処理を、Python の書き慣れた文法で実装できる」フレームワーク
- SQL と Python をシームレスに組み合わせたいデータエンジニア／データサイエンティストに最適なソリューション



In [ ]:
# 必要なライブラリや関数をインポート
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import col, max as max_
from snowflake.snowpark.context import get_active_session

# --- 1. Snowflakeセッションの取得 ---
# Snowflake Notebooksなどの環境からアクティブなセッションを直接取得します。
session = get_active_session()

# --- 2. メインの処理 ---
def main(session: Session):
    """
    指定されたSQLクエリをSnowparkで実装する関数
    """
    # SQL: FROM FINANCE__ECONOMICS.CYBERSYN.STOCK_PRICE_TIMESERIES
    source_df = session.table("FINANCE__ECONOMICS.CYBERSYN.STOCK_PRICE_TIMESERIES")

    # SQL: WHERE ticker = 'SNOW' AND variable_name IN ('Nasdaq Volume', 'Post-Market Close')
    filtered_df = source_df.filter(
        (col("ticker") == "SNOW") &
        (col("variable_name").in_("Nasdaq Volume", "Post-Market Close"))
    )
    
    # SQL (Subquery): SELECT ticker, date, variable_name, value
    # PIVOT操作の前に、必要な列だけを選択してGROUP BYキーを限定します。
    # これにより、同じ日付のデータが確実に1行にまとまります。
    pivot_input_df = filtered_df.select("ticker", "date", "variable_name", "value")

    # SQL: PIVOT(MAX(value) FOR variable_name IN ('Nasdaq Volume', 'Post-Market Close'))
    # PIVOT操作を実行します。
    pivoted_df = pivot_input_df.pivot(
        "variable_name", 
        ["Nasdaq Volume", "Post-Market Close"]
    ).agg(
        max_("value") # maxはPython組込み関数と衝突するためmax_としてインポート
    )

    # SQL: AS p (ticker, date, volume, price)
    # PIVOTで生成された列名（"'Nasdaq Volume'"など）を、希望の'volume', 'price'に変更します。
    # 同時に、PIVOTのキーとなった列も選択します。
    renamed_df = pivoted_df.select(
        col("ticker"),
        col("date"),
        col("'Nasdaq Volume'").alias("volume"),
        col("'Post-Market Close'").alias("price")
    )

    # SQL: ORDER BY date DESC
    # 最終的な結果を日付の降順で並べ替えます。
    final_df = renamed_df.order_by(col("date").desc())

    # 最終的なDataFrameの内容を表示して、クエリを実行します。
    print("変換後のデータ:")
    final_df.show()
    
    # 必要であれば、このDataFrameをテーブルに保存することも可能です。
    # final_df.write.mode("overwrite").save_as_table("YOUR_TARGET_TABLE_NAME")

    return "Processing complete."

# --- 3. スクリプトの実行 ---
if __name__ == '__main__':
    result = main(session)
    print(result)

### 2. 分析に使うデータを作成する

#### 株価、出来高、50日移動平均のデータをテーブル化
50日移動平均線のデータを作成し`SNOW_PRICE_VOLUME_50MA`テーブルに格納する

![img](https://lh3.googleusercontent.com/pw/AP1GczNuxR5J3sXIE6wq3YeTLbA1E_5smFxQs0SzdBLR1s-epVy_JOdl0LjgQW7Rzkgjkwu5vrFxudJSFgAJx3xRWnqaeitlFpXqH-ybUBzKBsERp7lR1GHmE777pKZrBKtJD-C-sMKVPX4uKaS3X9UMKhDP=w1337-h750-s-no-gm?authuser=0)

実装手順
- Snowflake（銘柄コード: SNOW）の出来高と終値を抽出し、日付ごとに見やすいように横並びの形式（ピボット）に変換して、最新の日付から順に表示
    - `Nasdaq Volume` → `volume`、`Post-Market Close` → `price` に列名を変更
- 上記テーブルから50日移動平均を算出し`movinng_avg_50`に格納

移動平均線とは過去の一定期間の終値の平均値を計算しそれを線で繋いだもの

![img](https://lh3.googleusercontent.com/pw/AP1GczM3inadA6wYhBX7umxHtzlcQqB99kdamVYS-b7pCO_n4bMcU0zb4EqVOxVx9f2xW8FUdply43v4XCkq5Ph14pVSHtsESqfGaeJRwV9Ygkh-8gMa3mqLo8w8z4bny6BIcz_4ZrAEJi3MkyauH6Mkp75e=w1303-h300-s-no-gm?authuser=0)

In [ ]:
-- SNOWの株価と出来高、50日移動平均を格納するテーブルを作成または置換する
CREATE OR REPLACE TABLE discover_handson.public.SNOW_PRICE_VOLUME_50MA AS
    -- ここからWITH句。共通テーブル式(CTE)で、まずデータを整形する
    WITH pivot_data AS (
        SELECT *
        FROM (
            -- PIVOTの元となるデータを準備
            SELECT 
                ticker,
                asset_class,
                primary_exchange_code,
                primary_exchange_name,
                date,
                variable_name,
                value
            FROM FINANCE__ECONOMICS.CYBERSYN.STOCK_PRICE_TIMESERIES 
            WHERE ticker = 'SNOW'
            -- 取得する指標を出来高と時間外終値に限定
            AND variable_name IN ('Nasdaq Volume', 'Post-Market Close')
        )
        -- PIVOT操作：縦持ちデータを横持ちに変換
        PIVOT(
            MAX(value) 
            FOR variable_name IN (
                'Nasdaq Volume',
                'Post-Market Close'
            )
        -- PIVOT句の直後でテーブルエイリアス(p)と新しい列名のリストを指定する
        ) AS p (ticker, asset_class, primary_exchange_code, primary_exchange_name, date, volume, price)
    )
    -- CTE 'pivot_data' を使って最終的なSELECT文を構成する
    SELECT 
        TICKER,
        DATE,
        volume,
        price,
        
        -- 移動平均の計算：ROWS BETWEENを使用して50営業日の移動平均を計算
        AVG(price) OVER (
            PARTITION BY ticker
            ORDER BY date 
            -- 過去50営業日（当日含む）の物理的な行数で計算
            ROWS BETWEEN 49 PRECEDING AND CURRENT ROW
        ) AS MOVING_AVG_50
    FROM pivot_data
    ORDER BY DATE DESC;

SELECT * FROM SNOW_PRICE_VOLUME_50MA ORDER BY DATE DESC;

#### 参考）Snowflake Pandas APIを使ってデータ加工と可視化を試す
- Snowpark pandas APIはSnowpark Pythonの一部として提供されるライブラリです。既存の pandas コードをほとんど書き換えずに、Snowflake 上で分散処理として実行できるように設計されています。

Snowpark DataFrame APIを選ぶ場合:
- データエンジニアやSQL開発者で、SQLの思考法に慣れている。
- Apache Spark (PySpark) の経験があり、似たようなAPIを使いたい。

Snowflake Pandas APIを選ぶ場合:
- Pandasの経験があり、その知識を活用したい。
- ローカルで開発したPandasのコードを、最小限の変更でSnowflakeのスケーラブルな環境に移行したい。

In [ ]:
# Snowflake Pandas APIを使ったSNOW株価データの基本分析
# ライブラリのインポート
import modin.pandas as pd
import snowflake.snowpark.modin.plugin
from snowflake.snowpark.context import get_active_session
import matplotlib.pyplot as plt

# データ読み込み
session = get_active_session()
pd.session = session 
df = pd.read_snowflake("discover_handson.public.SNOW_PRICE_VOLUME_50MA")

# データをソート
df = df.sort_values('DATE')
print(f"データ件数: {len(df)}件")

# === データ加工 ===
# 日次リターン（前日比変化率）を計算
df['daily_return'] = ((df['PRICE'] - df['PRICE'].shift(1)) / df['PRICE'].shift(1)) * 100

# 基本統計
print(f"平均価格: ${df['PRICE'].mean():.2f}")
print(f"平均日次リターン: {df['daily_return'].mean():.2f}%")

# === 可視化 ===
plt.figure(figsize=(15, 8))

# 1. 株価チャート
plt.subplot(2, 1, 1)
plt.plot(df['DATE'], df['PRICE'], label='SNOW Price', color='blue', linewidth=2)
plt.plot(df['DATE'], df['MOVING_AVG_50'], label='50-day Moving Average', color='red', linestyle='--', linewidth=2)
plt.title('SNOW Stock Price')
plt.ylabel('Price ($)')
plt.legend()
plt.grid(True, alpha=0.3)

# 2. 出来高チャート
plt.subplot(2, 1, 2)
plt.bar(df['DATE'], df['VOLUME'], color='green', alpha=0.7)
plt.title('Trading Volume')
plt.xlabel('Date')
plt.ylabel('Volume')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## ハンズオン3: Streamlit を使って加工したデータを可視化し示唆を得る

![img](https://lh3.googleusercontent.com/pw/AP1GczNwqh2Vf7qDYo9pXIWCQGiUhKiBfgKElSE01_JZTCYsGK2AeRb8ua6KLVvIpizHfMFcGT5tFOU7IqVXIRlTfvbV5YsIfwE1j3cAEJArZ0m7NdXH_j3pfEx7ZWJwdtN7D28Uj1_BgLLAC8Xi10LzQg_2=w2266-h1272-s-no-gm?authuser=0)

### Streamlit とは
- PythonのみでWebアプリを作成できるライブラリ
- HTML/CSSの知識不要でPythonスクリプトを直接Webアプリに変換

![img](https://lh3.googleusercontent.com/pw/AP1GczMhp7CFMU7LnrNkc6izujuwTnUDfVTkOAtAWF-pBQsOONqjsrsllpe7OVUn66AhcJNtommDtzUnHl0Los8H88rIqBz-n2yajpR9Z4HJzjxXrLb_6f77KRBUI4kQ9mnehPYUXDdA66OwsnYg2wwWuz_R=w1042-h617-s-no-gm?authuser=0)

#### 完成系のイメージ
![img](https://lh3.googleusercontent.com/pw/AP1GczNuhR_eUkra73UxZDgXcCY8AonaI-RGWe0tmGUAX008mmnO0X4wI4QC0x2b5ZsyrYfS-xoevPAyUA2QT6y61kYxqgWIGklkwDNuQXK1WhFFPiATprj3mKcwYf51wJC7GXsSWwTMfYVkQOdCUwGxbpUA=w1007-h741-s-no-gm?authuser=0)

### Step 1: 基本構造（タイトル・データ取得）
![img](https://lh3.googleusercontent.com/pw/AP1GczOMT9D85aFeWVgpC7TCu8aivCloYfSEkDCdpv-aGZtVXagOQdQUph4utvPo_XZy9XIRtiA6MXIv3XBfXGxUpz_leuq9DIr2_LNCvFIfR_wnCwYFC35RQgoFj3O3PDTLL1h_Ihs_40gfLLC-rJgjvjSV=w1013-h746-s-no-gm?authuser=0)
1. **`st.title()`** - メインタイトル表示
2. **`st.markdown()`** - リッチテキスト表示
3. **`@st.cache_data`** - データキャッシング
4. **基本的なデータ取得** - Snowflake接続

In [ ]:
# Step 1: 基本構造 - タイトルとデータ取得
# 完成系で使用する基本要素を学習

# 必要なパッケージのインポート
import streamlit as st
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from snowflake.snowpark.context import get_active_session
from datetime import date

# タイトル設定
st.title("📈 Snowflake (SNOW) 株価分析")
st.markdown("### Step 1: 基本構造を理解しよう")

# Snowflakeセッション取得
session = get_active_session()

# データキャッシュ関数
@st.cache_data
def get_stock_data(start_date, end_date):
    """株価データを取得する関数"""
    query = f"""
    SELECT * FROM SNOW_PRICE_VOLUME_50MA
    WHERE date >= '{start_date}' AND date <= '{end_date}'
    ORDER BY DATE ASC
    """
    result = session.sql(query)
    return result.to_pandas()

# デフォルト日付設定
default_start = date(2025, 2, 1)
default_end = date(2025, 4, 30)

df_stock_data = get_stock_data(default_start, default_end)

# データ確認
st.markdown("#### 取得したデータの確認")
st.write(f"期間: {default_start} ～ {default_end}")
st.write(f"データ行数: {len(df_stock_data)}")
st.dataframe(df_stock_data.head())

### Step 2: 日付選択とエラーハンドリング
![img](https://lh3.googleusercontent.com/pw/AP1GczPy_TixItaagD1f6Nfx18aDtHOFFDyqJtGzdOI_-1dD1Z_KMjk_KW4NiTkD-mlv2jrazDKBq4YA0wEPDvzaD6z4SITIDrTbTAYGEDlgAicISVLjDOiURYafQor68dXe0_t7SJhcfn3p2w4HhGtV64Iz=w1013-h737-s-no-gm?authuser=0)
1. **`st.date_input()`** - 日付選択ウィジェット
2. **条件分岐 (`if`文)** - エラーチェック
3. **`st.columns()`** = 画面を横に分割してコンテンツを並べて配置するレイアウト機能
4. **`st.metric()`** - 数値指標の見やすい表示

In [ ]:
# Step 2: 日付選択とエラーハンドリング
# 完成系で使用するインタラクティブ要素を学習

# カラム分割で日付選択を横並びに配置
col1, col2 = st.columns(2)

with col1:
    start_date = st.date_input(
        "開始日",
        value=date(2025, 2, 1)
    )

with col2:
    end_date = st.date_input(
        "終了日", 
        value=date(2025, 4, 30)
    )

# データ取得
with st.spinner(f"📊 {start_date} から {end_date} のデータを読み込み中..."):
    df_stock_data = get_stock_data(start_date, end_date)

# データ空チェック
if df_stock_data.empty:
    st.warning("⚠️ 選択した期間にはデータがありません。期間を変更してください。")
    st.stop()


# メトリクス計算
latest_price = df_stock_data['PRICE'].iloc[-1]
previous_price = df_stock_data['PRICE'].iloc[-2] if len(df_stock_data) > 1 else latest_price
price_change = ((latest_price / previous_price) - 1) * 100 if previous_price != 0 else 0
latest_ma = df_stock_data['MOVING_AVG_50'].iloc[-1]
latest_volume = df_stock_data['VOLUME'].iloc[-1]
avg_volume = df_stock_data['VOLUME'].mean()
volume_change_pct = ((latest_volume / avg_volume) - 1) * 100

# 4つのカラムでメトリクス表示
col1, col2, col3, col4 = st.columns(4)

with col1:
    st.metric(
        "直近株価",
        f"${latest_price:.2f}",
        f"{price_change:.2f}%" if len(df_stock_data) > 1 else "N/A"
    )

with col2:
    st.metric(
        "50日移動平均",
        f"${latest_ma:.2f}",
        f"{'移動平均線より上' if latest_price > latest_ma else '移動平均線より下'}"
    )

with col3:
    st.metric(
        "直近出来高",
        f"{latest_volume:,.0f}",
        f"{volume_change_pct:.1f}% vs 平均",
        delta_color="normal"
    )

with col4:
    st.metric("取引日数", len(df_stock_data))

### Step 3: メトリクス表示とカラムレイアウト
![img](https://lh3.googleusercontent.com/pw/AP1GczNdiXfSeQAxfvD_r-Glw35ef2AQtUgGw616KiSa6cvjr2hA_MGCpI2M5IY9aKCzcIYrvJaeBgn2lT_eUW-deY5txNnUgRpFX_CT6jN61Y9J7KD7i1ghziPRhEd7tnZNH1zixcDLjvc-11KhO9e1qa70=w1011-h745-s-no-gm?authuser=0)
1. **メトリクス計算** - 株価変化率、出来高比較などの計算
2. **`st.dataframe()`** - 統計テーブルの表示

In [ ]:
# Step 3: メトリクス表示とカラムレイアウト
# 完成系で使用する指標表示とレイアウトを学習

# 統計情報のカラム表示
st.markdown("### 📊 価格統計")
col1, col2 = st.columns(2)

with col1:
    st.markdown("#### 価格サマリー")
    price_stats = pd.DataFrame({
        '指標': ['現在価格', '平均価格', '最高値', '最安値', 'ボラティリティ（標準偏差）'],
        '値': [
            f"${df_stock_data['PRICE'].iloc[-1]:.2f}",
            f"${df_stock_data['PRICE'].mean():.2f}",
            f"${df_stock_data['PRICE'].max():.2f}",
            f"${df_stock_data['PRICE'].min():.2f}",
            f"${df_stock_data['PRICE'].std():.2f}"
        ]
    })
    st.dataframe(price_stats, hide_index=True, use_container_width=True)

with col2:
    st.markdown("#### 出来高サマリー")
    volume_stats = pd.DataFrame({
        '指標': ['直近出来高', '平均出来高', '最大出来高', '最小出来高', '累計出来高'],
        '値': [
            f"{df_stock_data['VOLUME'].iloc[-1]:,.0f}",
            f"{df_stock_data['VOLUME'].mean():,.0f}",
            f"{df_stock_data['VOLUME'].max():,.0f}",
            f"{df_stock_data['VOLUME'].min():,.0f}",
            f"{df_stock_data['VOLUME'].sum():,.0f}"
        ]
    })
    st.dataframe(volume_stats, hide_index=True, use_container_width=True)

### Step 4: Plotlyを使った可視化とExpander
![img](https://lh3.googleusercontent.com/pw/AP1GczPrHyPdluWwR9J3tPGmNobdoD0ysfv0kEKOMIkb15XnhGRYIxuh1gya7f8Q1EfmPqGB_OQwzLJ3P7eqRiiLt37Sbyu-dGhhXNrCGaH1SvqmQx0MtZBzzKlVPwUe4KwRgZ7PwPzGEsH7XJOBc5pDYhPh=w1012-h740-s-no-gm?authuser=0)
1. **`make_subplots()`** - 複数グラフの組み合わせ
2. **`go.Scatter()`** - ライングラフ作成
3. **`line=dict(dash='dash')`** - 破線スタイル
4. **`go.Bar()`** - バーグラフ作成
5. **グラフレイアウト設定** - ダークテーマ、軸設定
6. **`st.plotly_chart()`** - Plotlyグラフ表示
7. **`st.expander()`** - 折りたたみ表示
8. **`.style.format()`** - データフレーム書式設定

In [ ]:
# Step 4: Plotlyグラフとエキスパンダー
# 完成系で使用するグラフ表示と詳細データ表示を学習

#  Plotlyサブプロット作成
st.markdown("### 📈 株価チャート")

fig = make_subplots(
    rows=2, cols=1,
    row_heights=[0.7, 0.3],
    vertical_spacing=0.1,
    subplot_titles=("株価と50日移動平均線", "出来高"),
    shared_xaxes=True
)

# 株価ライングラフ
fig.add_trace(
    go.Scatter(
        x=df_stock_data['DATE'],
        y=df_stock_data['PRICE'],
        mode='lines',
        name='株価',
        line=dict(color='#00D9FF', width=2),
        hovertemplate='日付: %{x|%Y-%m-%d}<br>株価: $%{y:.2f}<extra></extra>'
    ),
    row=1, col=1
)

# 50日移動平均線
fig.add_trace(
    go.Scatter(
        x=df_stock_data['DATE'],
        y=df_stock_data['MOVING_AVG_50'],
        mode='lines',
        name='50日移動平均',
        line=dict(color='#FFA500', width=2, dash='dash'),
        hovertemplate='日付: %{x|%Y-%m-%d}<br>50日移動平均: $%{y:.2f}<extra></extra>'
    ),
    row=1, col=1
)

# 出来高バーグラフ（色分け）
colors = ['#FF4444' if i == 0 else '#FF4444' if df_stock_data['PRICE'].iloc[i] < df_stock_data['PRICE'].iloc[i-1] else '#00FF88' 
          for i in range(len(df_stock_data))]

fig.add_trace(
    go.Bar(
        x=df_stock_data['DATE'],
        y=df_stock_data['VOLUME'],
        name='出来高',
        marker_color=colors,
        opacity=0.8,
        hovertemplate='日付: %{x|%Y-%m-%d}<br>出来高: %{y:,.0f}<extra></extra>'
    ),
    row=2, col=1
)

# グラフレイアウト設定（ダークテーマ）
fig.update_layout(
    height=700,
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1,
        font=dict(color='white')
    ),
    hovermode='x unified',
    plot_bgcolor='black',
    paper_bgcolor='black',
    font=dict(color='white')
)

# 軸設定
fig.update_xaxes(
    title=dict(text="日付", font=dict(color='white')),
    row=2, col=1, 
    gridcolor='#333333',
    tickfont=dict(color='white')
)

fig.update_yaxes(
    title=dict(text="株価 ($)", font=dict(color='white')),
    row=1, col=1, 
    gridcolor='#333333',
    tickfont=dict(color='white')
)

fig.update_yaxes(
    title=dict(text="出来高", font=dict(color='white')),
    row=2, col=1, 
    gridcolor='#333333',
    tickfont=dict(color='white')
)

fig.update_annotations(font=dict(color='white'))

# グラフ表示
st.plotly_chart(fig, use_container_width=True)

# エキスパンダーでの生データ表示
with st.expander("📋 生データを表示", expanded=False):
    display_df = df_stock_data.sort_values('DATE', ascending=False).copy()
    display_df['DATE'] = pd.to_datetime(display_df['DATE']).dt.strftime('%Y-%m-%d')
    
    display_df = display_df.rename(columns={
        'DATE': '日付',
        'PRICE': '株価',
        'MOVING_AVG_50': '50日移動平均',
        'VOLUME': '出来高'
    })
    
    # フォーマット付きデータフレーム
    st.dataframe(
        display_df[['日付', '株価', '50日移動平均', '出来高']].style.format({
            '株価': '${:.2f}',
            '50日移動平均': '${:.2f}',
            '出来高': '{:,.0f}'
        }),
        use_container_width=True,
        height=400
    )

### Step 5: 完成系（全要素の組み合わせ）

In [ ]:
# 必要なパッケージのインポート
import streamlit as st
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from snowflake.snowpark.context import get_active_session
from datetime import date

# Snowflakeセッションの取得
session = get_active_session()

# 株価データと50日移動平均を取得する関数
@st.cache_data  # データをキャッシュして再読み込みを防ぐ
def get_stock_data(start_date, end_date):
    # PIVOTテーブルから50日移動平均を計算するシンプルなクエリ
    query = f"""
    SELECT * FROM SNOW_PRICE_VOLUME_50MA
    WHERE date >= '{start_date}' AND date <= '{end_date}'
    ORDER BY DATE ASC
    """
    
    # クエリを実行してPandasデータフレームに変換
    result = session.sql(query)
    return result.to_pandas()

# メインアプリケーションのタイトル設定
st.title("📈 Snowflake (SNOW) 株価分析")
st.markdown("### 期間を選択して株価、50日移動平均線、出来高を分析")
st.markdown("*RANGE関数を使用した正確なカレンダーベースの移動平均*")

# 日付範囲選択UI
col1, col2 = st.columns(2)
with col1:
    start_date = st.date_input(
        "開始日",
        value=date(2025, 2, 1),  # デフォルト値
        key="start_date"
    )
with col2:
    end_date = st.date_input(
        "終了日", 
        value=date(2025, 4, 30),  # デフォルト値
        key="end_date"
    )

# 日付の妥当性チェック
if start_date > end_date:
    st.error("開始日は終了日より前の日付を選択してください。")
    st.stop()

# データの読み込み（スピナーでローディング表示）
with st.spinner("株価データを読み込んでいます..."):
    df_stock_data = get_stock_data(start_date, end_date)

# データが空の場合の処理
if df_stock_data.empty:
    st.warning("選択した期間にはデータがありません。期間を変更してください。")
    st.stop()

# メトリクス表示用に4つのカラムを作成
col1, col2, col3, col4 = st.columns(4)

# 各種メトリクスの計算
latest_price = df_stock_data['PRICE'].iloc[-1]  # 最新の株価
previous_price = df_stock_data['PRICE'].iloc[-2] if len(df_stock_data) > 1 else latest_price  # 前日の株価
price_change = ((latest_price / previous_price) - 1) * 100 if previous_price != 0 else 0  # 価格変化率

# 第1カラム：直近株価と前日比
with col1:
    st.metric(
        "直近株価",
        f"${latest_price:.2f}",
        f"{price_change:.2f}%" if len(df_stock_data) > 1 else "N/A"
    )

# 第2カラム：50日移動平均と位置関係
with col2:
    latest_ma = df_stock_data['MOVING_AVG_50'].iloc[-1]  # 最新の50日移動平均
    st.metric(
        "50日移動平均",
        f"${latest_ma:.2f}",
        f"{'移動平均線より上' if latest_price > latest_ma else '移動平均線より下'}"
    )

# 第3カラム：出来高と平均比較
with col3:
    latest_volume = df_stock_data['VOLUME'].iloc[-1]  # 最新の出来高
    avg_volume = df_stock_data['VOLUME'].mean()  # 平均出来高
    volume_change_pct = ((latest_volume / avg_volume) - 1) * 100  # 出来高変化率
    st.metric(
        "直近出来高",
        f"{latest_volume:,.0f}",
        f"{volume_change_pct:.1f}% vs 平均",
        delta_color="normal"  # 正の値は緑、負の値は赤で表示
    )

# 第4カラム：取引日数
with col4:
    st.metric("取引日数", len(df_stock_data))

# 2段構成のグラフを作成（上段：株価、下段：出来高）
fig = make_subplots(
    rows=2, cols=1,
    row_heights=[0.7, 0.3],  # 上段70%、下段30%の高さ比率
    vertical_spacing=0.1,
    subplot_titles=("株価と50日移動平均線", "出来高"),
    shared_xaxes=True  # X軸を共有
)

# 株価の線グラフを追加
fig.add_trace(
    go.Scatter(
        x=df_stock_data['DATE'],
        y=df_stock_data['PRICE'],
        mode='lines',
        name='株価',
        line=dict(color='#00D9FF', width=2),  # 明るい青色
        hovertemplate='日付: %{x|%Y-%m-%d}<br>株価: $%{y:.2f}<extra></extra>'
    ),
    row=1, col=1  # 上段に配置
)

# 50日移動平均線を追加（破線）
fig.add_trace(
    go.Scatter(
        x=df_stock_data['DATE'],
        y=df_stock_data['MOVING_AVG_50'],
        mode='lines',
        name='50日移動平均',
        line=dict(color='#FFA500', width=2, dash='dash'),  # オレンジ色の破線
        hovertemplate='日付: %{x|%Y-%m-%d}<br>50日移動平均: $%{y:.2f}<extra></extra>'
    ),
    row=1, col=1  # 上段に配置
)

# 出来高バーの色を価格変動に基づいて設定（下落：赤、上昇：緑）
colors = ['#FF4444' if i == 0 else '#FF4444' if df_stock_data['PRICE'].iloc[i] < df_stock_data['PRICE'].iloc[i-1] else '#00FF88' 
          for i in range(len(df_stock_data))]

# 出来高の棒グラフを追加
fig.add_trace(
    go.Bar(
        x=df_stock_data['DATE'],
        y=df_stock_data['VOLUME'],
        name='出来高',
        marker_color=colors,
        opacity=0.8,
        hovertemplate='日付: %{x|%Y-%m-%d}<br>出来高: %{y:,.0f}<extra></extra>'
    ),
    row=2, col=1  # 下段に配置
)

# グラフのレイアウト設定（ダークテーマ）
fig.update_layout(
    height=700,  # グラフの高さ
    showlegend=True,
    legend=dict(
        orientation="h",  # 水平方向
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1,
        font=dict(color='white')
    ),
    hovermode='x unified',  # X軸で統一されたホバー表示
    plot_bgcolor='black',  # プロット背景色
    paper_bgcolor='black',  # 全体背景色
    font=dict(color='white')  # フォント色
)

# X軸の設定（日付）
fig.update_xaxes(
    title=dict(text="日付", font=dict(color='white')),
    row=2, col=1, 
    gridcolor='#333333',  # グリッド線の色
    tickfont=dict(color='white')
)

# Y軸の設定（株価）
fig.update_yaxes(
    title=dict(text="株価 ($)", font=dict(color='white')),
    row=1, col=1, 
    gridcolor='#333333',
    tickfont=dict(color='white')
)

# Y軸の設定（出来高）
fig.update_yaxes(
    title=dict(text="出来高", font=dict(color='white')),
    row=2, col=1, 
    gridcolor='#333333',
    tickfont=dict(color='white')
)

# サブプロットタイトルの色を白に設定
fig.update_annotations(font=dict(color='white'))

# グラフを表示
st.plotly_chart(fig, use_container_width=True)

# 展開可能な生データ表示セクション
with st.expander("📋 生データを表示", expanded=False):
    # データを新しい順（降順）にソート
    display_df = df_stock_data.sort_values('DATE', ascending=False).copy()
    # 日付フォーマットを変換
    display_df['DATE'] = pd.to_datetime(display_df['DATE']).dt.strftime('%Y-%m-%d')
    
    # カラム名を日本語に変更
    display_df = display_df.rename(columns={
        'DATE': '日付',
        'PRICE': '株価',
        'MOVING_AVG_50': '50日移動平均',
        'VOLUME': '出来高'
    })
    
    # フォーマットを適用してデータフレームを表示
    st.dataframe(
        display_df[['日付', '株価', '50日移動平均', '出来高']].style.format({
            '株価': '${:.2f}',
            '50日移動平均': '${:.2f}',
            '出来高': '{:,.0f}'
        }),
        use_container_width=True,
        height=400
    )

# 価格統計セクション
st.markdown("### 📊 価格統計")

# 2カラムレイアウトで統計情報を表示
col1, col2 = st.columns(2)

# 左側：価格サマリー
with col1:
    st.markdown("#### 価格サマリー")
    # 価格統計のデータフレームを作成
    price_stats = pd.DataFrame({
        '指標': ['現在価格', '平均価格', '最高値', '最安値', 'ボラティリティ（標準偏差）'],
        '値': [
            f"${df_stock_data['PRICE'].iloc[-1]:.2f}",
            f"${df_stock_data['PRICE'].mean():.2f}",
            f"${df_stock_data['PRICE'].max():.2f}",
            f"${df_stock_data['PRICE'].min():.2f}",
            f"${df_stock_data['PRICE'].std():.2f}"
        ]
    })
    st.dataframe(price_stats, hide_index=True, use_container_width=True)

# 右側：出来高サマリー
with col2:
    st.markdown("#### 出来高サマリー")
    # 出来高統計のデータフレームを作成
    volume_stats = pd.DataFrame({
        '指標': ['直近出来高', '平均出来高', '最大出来高', '最小出来高', '累計出来高'],
        '値': [
            f"{df_stock_data['VOLUME'].iloc[-1]:,.0f}",
            f"{df_stock_data['VOLUME'].mean():,.0f}",
            f"{df_stock_data['VOLUME'].max():,.0f}",
            f"{df_stock_data['VOLUME'].min():,.0f}",
            f"{df_stock_data['VOLUME'].sum():,.0f}"
        ]
    })
    st.dataframe(volume_stats, hide_index=True, use_container_width=True)



## ハンズオン4: LLM を使った分析
- AISQL を使った分析を試していく（詳細は後半のハンズオンにて）
![img](https://lh3.googleusercontent.com/pw/AP1GczPEstMLsFYzhztvUUHigYdBALM3BDLmvKUQKWDMmCQKuebzruOhKZpKoZeMH1kcoVBil8SUwscciE4_z4AtPx6UrRPwfLt6nhJUS1Lg-q8arZy_TAW6-ZYLdGplQedh7XQxLIgcfYobF6Lk8jLAcn10=w1149-h646-s-no-gm?authuser=0)

AISQLを試してみる
- AI_COMPLETE, AI_AGGを活用した分析を試す
- (AI_AGGはAppendix部分に記載)

使い方を理解する

In [ ]:
SELECT AI_COMPLETE ('claude-3-7-sonnet', 'Snowflakeの特徴を50字以内で教えてください。');

### Streamlitで可視化した結果の画像として分析

以下の画像をAI_COMPLETE関数を使って分析してみる

![img](https://lh3.googleusercontent.com/pw/AP1GczP5WFrRw4SEs1rOL6-U-Scgrq-i4f9VS4eDut7F6bDDiEsn2uPHrecdybPE-hQxcZwWj_Ekymhi0kpuEVOPpnkj8K-qZY2sjXC8dR8a6dBeiPMzKDlxlzNzHNtJHwLJxxdiq_t9shpyy6oArjs5cqsa=w1149-h646-s-no-gm?authuser=0)


In [ ]:
-- 30s弱
SELECT *, AI_COMPLETE('claude-3-7-sonnet','入力されたSnowflakeの株価と出来高の日足の画像データからどのような傾向があるかを分析してください。', to_file('@handson_stage', 'SNOW_img.png')) as EXTRACT_DATA;

In [ ]:
import streamlit as st
st.table(image_analysis.to_pandas()["EXTRACT_DATA"])

### テキストデータを分析する
- SEC_REPORT_TEXT_ATTRIBUTESのデータを活用

    - 8-K (Current Report / 臨時報告書)
        - 決算発表のような株価に影響を与えうる重要な出来事があった際、この「Form 8-K」をSEC（米国証券取引委員会）に提出する義務
        - 8-K Filing Text:
            - 企業の重要な出来事を随時報告するための書類の本編。M&A、経営陣の交代、倒産、重要な契約の締結など、株価に大きな影響を与える可能性のある出来事が発生した際に提出される。
        - 8-K EX-99 Filing Text:
            - 8-Kに添付される補足資料。多くの場合、その出来事に関するプレスリリースや投資家向け説明資料などがここに添付され法律で定められた形式以外の自由な形式の開示に利用。

    - 10-K（Annual Report / 年次報告書）
        - 年に一度提出される、企業の通期の業績報告書です。10-Qと異なり、公認会計士による監査を受けており、最も包括的で信頼性の高い情報源です。
    - 10-Q (Quarterly Report / 四半期報告書)
        - 10-Q Filing Text:
            - 企業の四半期ごとの業績を報告する書類の本編。財務諸表（貸借対照表、損益計算書など）や経営陣による業績の分析が含まれる。
        
    など

![img](https://lh3.googleusercontent.com/pw/AP1GczPkCu_MArISFcX9hRejbCKKfcPRrL_hpU9f6KrpI48QozTf84YbPkX2NITGelp5bz-HsQ-li5rJdKA203Rb48zjtPfONRc4R7bGIpYdF1ATh-nS-l9umtlCpVziGLWlmuZnUYAK4ePGe-3KywQ1RJN-=w2262-h1268-s-no-gm?authuser=0)



In [ ]:
-- SEC報告書のテキスト属性データのサンプルを10件取得
SELECT *
FROM SEC_FILINGS.CYBERSYN.SEC_REPORT_TEXT_ATTRIBUTES
LIMIT 10;

Snowflake のデータを探す
- CYBERSYN.COMPANY_INDEXデータから社名に`Snowflake`を含む企業を検索

In [ ]:
SELECT *
FROM SEC_FILINGS.CYBERSYN.COMPANY_INDEX
WHERE company_name ILIKE '%Snowflake%';

CIK =`0001640147`を指定しSnowflakeに関する情報を抽出

In [ ]:
SELECT 
    *
FROM SEC_FILINGS.CYBERSYN.SEC_REPORT_TEXT_ATTRIBUTES
WHERE CIK = 0001640147
ORDER BY period_end_date DESC
LIMIT 20;

直近の10-Q (Quarterly Report / 四半期報告書)を情報からAI_COMPLETE関数を使って重要な情報を抽出する

In [ ]:
-- 最新10-Qから重要な情報を抽出
SELECT 
    AI_COMPLETE(
        'claude-3-7-sonnet',
        CONCAT('この四半期報告書から重要な財務情報とビジネスハイライトを日本語で要約してください。: <10q_document>', VALUE, '</10q_document>')
    ) AS financial_summary,
    PERIOD_END_DATE,
    VARIABLE_NAME
FROM SEC_FILINGS.CYBERSYN.SEC_REPORT_TEXT_ATTRIBUTES 
WHERE CIK = '0001640147'
    AND VARIABLE = '10-Q_Filing_Text'
ORDER BY PERIOD_END_DATE DESC 
LIMIT 1;

ファンダメンタル情報を付与したデータの作成
-  企業の重要な出来事による株価の影響を分析する
    - 8-K(企業の重要な出来事を随時報告するための書類)提出日後の株価動向を調べる

![img](https://lh3.googleusercontent.com/pw/AP1GczNgnYh01d0IMnBZxG-ve5nUka5g1jz9pElQBqjcMZ2xP28lRw6JidgnV_jkVphS0fyQr5xIcD9ULoYVZsIH9r46J8kO8Pce9y0ldEmY7BS2Qg2Sg9mS5ofnlpzRWYNsM5gsgm8gzGDep3jKuXfq7t87=w2262-h1262-s-no-gm?authuser=0)

In [ ]:
CREATE OR REPLACE TABLE SNOW_PRICE_WITH_FILINGS AS

-- ステップ1: 必要な8-Kレポートだけをフィルタリングし、PIVOTを使って横持ち形式に変換するCTEを作成
WITH pivoted_filings AS (
  SELECT
    *
  FROM (
    -- 結合に必要なカラムだけを選択し、8-Kレポートのみに絞る
    SELECT
      PERIOD_END_DATE,
      VARIABLE_NAME,
      VALUE
    FROM SEC_FILINGS.CYBERSYN.SEC_REPORT_TEXT_ATTRIBUTES
    WHERE CIK = '0001640147'
        AND VARIABLE_NAME IN ('8-K Filing Text', '8-K EX-99 Filing Text')
  )
  PIVOT(
    -- VALUE列の値を集約する（同じ日に同じ種類のレポートは1つしかないのでMAXで問題ない）
    MAX(VALUE)
    -- VARIABLE_NAME列の値を使って新しいカラム名を作成する
    FOR VARIABLE_NAME IN ('8-K Filing Text', '8-K EX-99 Filing Text')
  ) AS p (
    -- PIVOT後のカラム名を定義する
    PERIOD_END_DATE,
    "8-K_Filing_Text_VALUE",      -- カラム名に特殊文字が含まれるためダブルクォートで囲む
    "8-K_EX-99_Filing_Text_VALUE" -- 同上
  )
)

-- ステップ2: 株価テーブルを基準に、横持ちにした8-KレポートをLEFT JOINで結合
SELECT
  p.*,  -- 株価テーブルの全カラムを表示
  pf."8-K_Filing_Text_VALUE",
  pf."8-K_EX-99_Filing_Text_VALUE"
FROM SNOW_PRICE_VOLUME_50MA AS p
LEFT JOIN pivoted_filings AS pf
  ON p.DATE = pf.PERIOD_END_DATE
ORDER BY p.DATE DESC;

SELECT * FROM SNOW_PRICE_WITH_FILINGS ORDER BY date DESC;

8-k提出前後の株価反応分析

In [ ]:
-- ステップ1: 8-Kレポートが提出された日をSNOW_PRICE_WITH_FILINGSテーブルから特定
WITH event_dates AS (
    SELECT DISTINCT
        DATE AS event_date
    FROM SNOW_PRICE_WITH_FILINGS
    WHERE ("8-K_Filing_Text_VALUE" IS NOT NULL 
           OR "8-K_EX-99_Filing_Text_VALUE" IS NOT NULL)
),

-- ステップ2: 8-Kレポートの内容概要を取得
filing_content AS (
    SELECT 
        DATE as event_date,
        CASE 
            WHEN "8-K_Filing_Text_VALUE" IS NOT NULL THEN 
                LEFT("8-K_Filing_Text_VALUE", 100) || '...'
            ELSE NULL
        END as filing_summary,
        CASE 
            WHEN "8-K_EX-99_Filing_Text_VALUE" IS NOT NULL THEN 
                LEFT("8-K_EX-99_Filing_Text_VALUE", 100) || '...'
            ELSE NULL
        END as ex99_summary
    FROM SNOW_PRICE_WITH_FILINGS
    WHERE ("8-K_Filing_Text_VALUE" IS NOT NULL 
           OR "8-K_EX-99_Filing_Text_VALUE" IS NOT NULL)
),

-- ステップ3: 各8-Kイベント日の前後2日の株価データと分析指標を計算
price_reaction AS (
    SELECT 
        p.DATE,
        p.PRICE,
        p.VOLUME,
        p.MOVING_AVG_50,
        e.event_date,
        -- ウィンドウ関数は必ずイベント日ごとにPARTITION BYで区切り、計算が混ざらないようにする
        LAG(p.PRICE, 1) OVER (PARTITION BY e.event_date ORDER BY p.DATE) as prev_close,
        AVG(p.VOLUME) OVER (PARTITION BY e.event_date ORDER BY p.DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING) as avg_volume_20d,
        DATEDIFF('day', e.event_date, p.DATE) as days_from_event
    FROM SNOW_PRICE_WITH_FILINGS p
    -- 各イベント日に対して、その前後の株価データを紐付ける
    CROSS JOIN event_dates e
    WHERE p.DATE BETWEEN DATEADD('day', -2, e.event_date) AND DATEADD('day', 2, e.event_date)
)

-- ステップ4: 分析結果を見やすく整形して出力
SELECT 
    -- 日付と曜日を日本語で表示
    TO_CHAR(pr.DATE, 'YYYY/MM/DD') || 
    CASE DAYOFWEEK(pr.DATE)
        WHEN 0 THEN '（日）' WHEN 1 THEN '（月）' WHEN 2 THEN '（火）'
        WHEN 3 THEN '（水）' WHEN 4 THEN '（木）' WHEN 5 THEN '（金）'
        WHEN 6 THEN '（土）'
    END as "日付",
    
    pr.days_from_event as "イベントからの日数",
    ROUND(pr.PRICE, 2) as "終値",
    
    -- 前日比変動率
    CASE 
        WHEN pr.prev_close IS NOT NULL THEN
            CASE WHEN pr.PRICE > pr.prev_close THEN '+' ELSE '' END ||
            ROUND((pr.PRICE - pr.prev_close) / pr.prev_close * 100, 2) || '%'
        ELSE 'N/A'
    END as "前日比",
    
    -- 出来高情報
    ROUND(pr.VOLUME / 1000000, 2) as "出来高_百万株",
    CASE 
        WHEN pr.avg_volume_20d > 0 THEN ROUND((pr.VOLUME - pr.avg_volume_20d) / pr.avg_volume_20d * 100, 0)
        ELSE NULL
    END as "出来高変化率％",
    
    -- 50日移動平均線との乖離
    ROUND((pr.PRICE - pr.MOVING_AVG_50) / pr.MOVING_AVG_50 * 100, 2) as "50日MA乖離率％",
    
    -- 期間分類
    CASE 
        WHEN pr.days_from_event = 0 THEN '★8-K提出日★'
        WHEN pr.days_from_event = -1 THEN '提出前日'
        WHEN pr.days_from_event = 1 THEN '提出翌日'
        WHEN pr.days_from_event = -2 THEN '提出2日前'
        WHEN pr.days_from_event = 2 THEN '提出2日後'
        ELSE '周辺期間'
    END as "期間分類",
    
    -- 8-Kレポートの内容概要（提出日のみ表示）
    CASE 
        WHEN pr.days_from_event = 0 THEN fc.filing_summary
        ELSE NULL
    END as "8-K概要",
    
    CASE 
        WHEN pr.days_from_event = 0 THEN fc.ex99_summary
        ELSE NULL
    END as "EX-99概要"
    
FROM price_reaction pr
LEFT JOIN filing_content fc ON pr.event_date = fc.event_date
WHERE pr.days_from_event BETWEEN -2 AND 2
ORDER BY pr.event_date DESC, pr.DATE ASC;

株価が急上昇した要因となるファンダメンタル情報の抽出

In [ ]:
-- ============================================================
-- 株価急上昇判定とLLMを使った分析
-- Snowflake Cortex AI_COMPLETEを使用した8-K提出後の株価急上昇分析
-- ============================================================

-- Step 1: 8-K提出日の特定と基本データ準備
WITH filing_events AS (
    SELECT 
        p.DATE,
        p.PRICE,
        p.VOLUME,
        p.MOVING_AVG_50,
        CASE 
            WHEN "8-K_Filing_Text_VALUE" IS NOT NULL 
                 OR "8-K_EX-99_Filing_Text_VALUE" IS NOT NULL 
            THEN 1 
            ELSE 0 
        END as is_filing_day,
        CASE 
            WHEN "8-K_EX-99_Filing_Text_VALUE" IS NOT NULL THEN 
                LEFT("8-K_EX-99_Filing_Text_VALUE", 1500)
            ELSE NULL
        END as ex99_text,
        
        -- 前日終値
        LAG(p.PRICE, 1) OVER (ORDER BY p.DATE) as prev_close,
        
        -- 20日平均出来高
        AVG(p.VOLUME) OVER (ORDER BY p.DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING) as avg_volume_20d
        
    FROM SNOW_PRICE_WITH_FILINGS p
    WHERE p.DATE >= '2025-05-01'
),

-- Step 2: 急上昇判定と指標計算
surge_detection AS (
    SELECT 
        fe.*,
        
        -- 価格変動率
        CASE 
            WHEN fe.prev_close IS NOT NULL AND fe.prev_close > 0 THEN
                ROUND((fe.PRICE - fe.prev_close) / fe.prev_close * 100, 2)
            ELSE NULL
        END as price_change_pct,
        
        -- 出来高変化率
        CASE 
            WHEN fe.avg_volume_20d > 0 THEN 
                ROUND((fe.VOLUME - fe.avg_volume_20d) / fe.avg_volume_20d * 100, 0)
            ELSE NULL
        END as volume_change_pct,
        
        -- 50日移動平均からの乖離率
        CASE 
            WHEN fe.MOVING_AVG_50 > 0 THEN
                ROUND((fe.PRICE - fe.MOVING_AVG_50) / fe.MOVING_AVG_50 * 100, 2)
            ELSE NULL
        END as ma50_deviation_pct,
        
        -- 急上昇判定フラグ
        CASE 
            WHEN fe.is_filing_day = 1 AND  
                 fe.prev_close IS NOT NULL AND fe.prev_close > 0 AND
                 ((fe.PRICE - fe.prev_close) / fe.prev_close * 100) >= 5.0 AND  
                 fe.avg_volume_20d > 0 AND
                 ((fe.VOLUME - fe.avg_volume_20d) / fe.avg_volume_20d * 100) >= 50.0  
            THEN 'SURGE_DETECTED'
            ELSE 'NO_SURGE'
        END as surge_flag,
        
        -- 急上昇強度レベル
        CASE 
            WHEN fe.is_filing_day = 1 AND fe.prev_close IS NOT NULL AND fe.prev_close > 0 THEN
                CASE 
                    WHEN ((fe.PRICE - fe.prev_close) / fe.prev_close * 100) >= 15.0 THEN 'VERY_HIGH'
                    WHEN ((fe.PRICE - fe.prev_close) / fe.prev_close * 100) >= 10.0 THEN 'HIGH'
                    WHEN ((fe.PRICE - fe.prev_close) / fe.prev_close * 100) >= 5.0 THEN 'MODERATE'
                    ELSE 'LOW'
                END
            ELSE NULL
        END as surge_intensity
        
    FROM filing_events fe
)

-- Step 3: 最終出力
SELECT 
    TO_CHAR(sd.DATE, 'YYYY/MM/DD') || 
    CASE DAYOFWEEK(sd.DATE)
        WHEN 0 THEN '（日）' WHEN 1 THEN '（月）' WHEN 2 THEN '（火）'
        WHEN 3 THEN '（水）' WHEN 4 THEN '（木）' WHEN 5 THEN '（金）'
        WHEN 6 THEN '（土）'
    END as "日付",
    
    ROUND(sd.PRICE, 2) as "終値",
    
    CASE 
        WHEN sd.price_change_pct IS NOT NULL THEN
            CASE WHEN sd.price_change_pct > 0 THEN '+' ELSE '' END ||
            sd.price_change_pct || '%'
        ELSE 'N/A'
    END as "前日比",
    
    ROUND(sd.VOLUME / 1000000, 2) as "出来高_百万株",
    sd.volume_change_pct as "出来高変化率％",
    sd.ma50_deviation_pct as "50日MA乖離率％",
    
    sd.surge_flag as "急上昇判定",
    sd.surge_intensity as "急上昇強度",
    
    CASE 
        WHEN sd.is_filing_day = 1 THEN '★8-K提出日★'
        ELSE '通常日'
    END as "期間分類",
    
    -- LLM分析（急上昇検出時のみ実行）
    CASE 
        WHEN sd.surge_flag = 'SURGE_DETECTED' AND 
             sd.ex99_text IS NOT NULL AND
             LENGTH(sd.ex99_text) > 100 THEN
             
            AI_COMPLETE(
                'claude-3-7-sonnet',
                'この8-K EX-99文書から株価上昇の主要因を3つ箇条書きで説明してください：' || 
                SUBSTR(sd.ex99_text, 1, 1000)
            )
        ELSE NULL
    END as "LLMの分析結果",
    CASE 
        WHEN sd.is_filing_day = 1 AND sd.ex99_text IS NOT NULL THEN 
            LEFT(sd.ex99_text, 150) || '...'
        ELSE NULL
    END as "EX-99文書抜粋"
    
FROM surge_detection sd
WHERE sd.is_filing_day = 1  -- 8-K提出日のみ表示
   OR sd.surge_flag = 'SURGE_DETECTED'  -- または急上昇日
ORDER BY sd.DATE DESC;

## Appendix

Form 8-K（臨時報告書）から企業ストーリーを辿る

In [ ]:
SELECT 
    AI_AGG(VALUE, '「提供された全Form 8-Kのレコードを分析し、Snowflakeの過去の『企業ストーリー』をまとめてください。このストーリーには、以下の要素を必ず含めて日本語で回答してください。」
                    成長期: M&Aや好調な決算発表によって事業が急拡大した期間。
                    転換期: CEOの交代や大規模な戦略変更など、会社の方向性が大きく変わった重要なイベント。
                    挑戦期: 業績不振、リストラ、重大な訴訟など、会社が困難に直面した期間。
                    戦略の進化: 時間の経過と共に、会社の事業の焦点（例：製品、ターゲット市場）がどのように変化したか。
                    資本政策の変化: 自社株買いや増資など、株主還元や資金調達に関する方針がどう変わってきたか。') AS analysis_result
FROM (
    SELECT 
        PERIOD_END_DATE,
        VALUE
    FROM SEC_FILINGS.CYBERSYN.SEC_REPORT_TEXT_ATTRIBUTES 
    WHERE CIK = 0001640147 
        AND VARIABLE = '8-K_Filing_Text' 
    ORDER BY PERIOD_END_DATE DESC
    LIMIT 5
);

8-K EX-99（プレスリリース)から企業の変遷を知る
- 報告書の「具体的な内容」であり、本体の報告書に添付される資料
- その四半期の売上高、純利益、1株当たり利益（EPS）、経営陣によるコメント、今後の業績見通し（ガイダンス）、詳細な財務データ表など、市場が最も注目する情報が含まれている

In [ ]:
SELECT 
    AI_AGG(VALUE, '「このEX-99（プレスリリース）を投資アナリストの視点で分析し、今後の株価に対する『強気（Bull）シナリオの根拠』と『弱気（Bear）シナリオの根拠』をそれぞれ3つずつ、具体的な記述を引用しながら要約してください。」
                出力形式:
                【強気（Bull）シナリオの根拠】
                理由: （例：新製品の需要が好調）
                根拠となる引用: 「"Our new AI-powered platform has seen unprecedented adoption rates..."」
                ...
                【弱気（Bear）シナリオの根拠】
                理由: （例：マクロ経済の不確実性による顧客の予算削減懸念）
                根拠となる引用: 「"We remain cautious given the macroeconomic uncertainty that may impact customer spending..."」
                ...') AS analysis_result
FROM (
    SELECT 
        PERIOD_END_DATE,
        VALUE
    FROM SEC_FILINGS.CYBERSYN.SEC_REPORT_TEXT_ATTRIBUTES 
    WHERE CIK = 0001640147 
        AND VARIABLE = '8-K_EX-99_Filing_Text' 
    ORDER BY PERIOD_END_DATE DESC
    LIMIT 2
);